## LDA를 이용한 토픽 모델링
LDA를 통해서 강의계획서를 군집화하고, 그 결과를 이용해 유사한 강의를 추출할 수 있도록 하는 것이 목표.
1. 텍스트를 담은 데이터프레임 만들기
2. 정규표현식으로 데이터 클리닝
3. konlpy의 품사 태깅 -> 명사만 추출(?)
4. 벡터화 결과를 이용해 LDA 진행

In [215]:
import pandas as pd

In [216]:
DATA_IN_PATH = "C:\\Users\\dmb08\\Desktop\\kucc_ycc_hack\\"
File_Name = "문과대학_이과대학.csv"

data = pd.read_csv(DATA_IN_PATH + File_Name, encoding='CP949')
data

,강의명,수업목표
0,문학의 이해,국문과에 개설된 <문학이란 무엇인가?> 과이 문학에 대한 기본적 이해와 한국문학의 ...
1,한국미술사,<한국미술사>에서는 한국회화사에서 논의되는 ‘걸작’을 중점적으로 강의합니다. 특히 ...
2,서양철학의 이해,본 수업의 목표는 서양철학사의 전반적인 문제와 흐름에 대한 기본적인 이해를 돕는 것...
3,현대사회와정신건강,현대사회에서 신체적 건강뿐만 아니라 정신적 건강의 중요성이 더욱 부각되고 있는 시점...
4,언어의세계,인간은 언어적 존재다. 인간의 핵심에 언어가 있다. 그런데 인간만 언어를 갖고 있는...
5,사회학의 이해,이 강의는 사회학의 개론을 다루는 교양 강의입니다.\n별도 선수과목은 요구하지 않으...
6,수학사,구석기 시대부터 기원전 수천 년에 걸친 문명의 탄생과 고대로부터 현대에 이르기까지 ...
7,편미분방정식,편미분 방정식은 자연현상을 수학적으로 기술한 형태중 편미분을 포함한 경우에 해당하는...
8,수학논리및논술?,"본 수업에서는 학교 수학에 포섭된 수학 논리 논술과 관련한 선행 논의들을 검토하고,..."


In [217]:
from konlpy.tag import Okt
import konlpy
from tqdm import tqdm
import re
import pickle
import csv
import pandas as pd
from pandas import DataFrame 
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [218]:
words = pd.read_csv('E:\\김밥과 메추리알\\고려대학교\\3-1\\데이터마이닝\\데마 코드 최종\\데이터셋\\stopwords.csv', header=None)
stopdicts = list(words.iloc[0])

In [219]:
from konlpy.tag import Okt

def text_cleaning(text):
    # 한글의 정규표현식으로 한글만 추출합니다.
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    result = hangul.sub('', text)
    return result

def get_nouns(tokenizer, sentence):
    
    #품사태깅 후 조사 및 불필요 형태소 제거
    tagged = tokenizer.pos(sentence)
    words = [s for s,t in tagged if t not in ['JKS','JKC','JKG','JKO','JKB','JKV','JX','JC','UNKNOWN','EC','XSA','VA', 'VV','IC','EF','ETM','XSN','','Josa'] and len(s)>0]
    
    #정규화
    verbs = [s+'다' for s,t in tagged if t in ['VA', 'VV']]
    words = words+verbs
    
    #불용어 제거 & remove 1 word
    stopwords = stopdicts + ['너무너무','입니다','셔서','히다','그래서인지','로웠','성역','시빌','대도','세요','해서','합니다','할게요','건지','비갑','목표','하며','하여','과목','관련']
    words = [i for i in words if i not in stopwords and (len(i)>1)]
    
    return words
  

def tokenize(df):
    tokenizer = Okt()
    processed_data = []
    
    #정규표현식을 이용해 한글만 추출
    df['수업목표'] = df['수업목표'].apply(lambda x: text_cleaning(x))
    df = df[df['수업목표'].str.len() > 0]
    df.dropna()
    
    for sent in tqdm(df['수업목표']):
        processed_data.append(get_nouns(tokenizer, sent))
    return processed_data
  
if __name__ == '__main__':
    df = data
    df.dropna(how='any')
    processed_data = tokenize(df)
    data['tagged'] = processed_data

100%|██████████| 9/9 [00:00<00:00, 67.65it/s]


In [220]:
data.head()

,강의명,수업목표,tagged
0,문학의 이해,국문과에 개설된 문학이란 무엇인가 과이 문학에 대한 기본적 이해와 한국문학의 흐름에...,"[국문과, 개설, 문학, 문학, 기본, 이해, 국문학, 흐름, 개괄, 전교생, 대상..."
1,한국미술사,한국미술사에서는 한국회화사에서 논의되는 걸작을 중점적으로 강의합니다 특히 조선시대부...,"[한국, 미술사, 국회, 화사, 논의, 되는, 걸작, 중점, 강의, 조선시대, 세기..."
2,서양철학의 이해,본 수업의 목표는 서양철학사의 전반적인 문제와 흐름에 대한 기본적인 이해를 돕는 것...,"[수업, 서양철학, 사의, 전반, 문제, 흐름, 기본, 이해, 돕는, 이를, 고대,..."
3,현대사회와정신건강,현대사회에서 신체적 건강뿐만 아니라 정신적 건강의 중요성이 더욱 부각되고 있는 시점...,"[사회, 신체, 건강, 아니라, 정신, 건강, 중요성, 더욱, 부각, 되고, 있는,..."
4,언어의세계,인간은 언어적 존재다 인간의 핵심에 언어가 있다 그런데 인간만 언어를 갖고 있는 것...,"[인간, 언어, 존재, 인간, 핵심, 언어, 인간, 언어, 갖고, 있는, 세계, 전..."


In [221]:
data['tagged'].to_csv('C:\\Users\\dmb08\\Desktop\\kucc_ycc_hack\\token_tag.csv',  header = False, index = False, encoding='CP949')

In [222]:
data

,강의명,수업목표,tagged
0,문학의 이해,국문과에 개설된 문학이란 무엇인가 과이 문학에 대한 기본적 이해와 한국문학의 흐름에...,"[국문과, 개설, 문학, 문학, 기본, 이해, 국문학, 흐름, 개괄, 전교생, 대상..."
1,한국미술사,한국미술사에서는 한국회화사에서 논의되는 걸작을 중점적으로 강의합니다 특히 조선시대부...,"[한국, 미술사, 국회, 화사, 논의, 되는, 걸작, 중점, 강의, 조선시대, 세기..."
2,서양철학의 이해,본 수업의 목표는 서양철학사의 전반적인 문제와 흐름에 대한 기본적인 이해를 돕는 것...,"[수업, 서양철학, 사의, 전반, 문제, 흐름, 기본, 이해, 돕는, 이를, 고대,..."
3,현대사회와정신건강,현대사회에서 신체적 건강뿐만 아니라 정신적 건강의 중요성이 더욱 부각되고 있는 시점...,"[사회, 신체, 건강, 아니라, 정신, 건강, 중요성, 더욱, 부각, 되고, 있는,..."
4,언어의세계,인간은 언어적 존재다 인간의 핵심에 언어가 있다 그런데 인간만 언어를 갖고 있는 것...,"[인간, 언어, 존재, 인간, 핵심, 언어, 인간, 언어, 갖고, 있는, 세계, 전..."
5,사회학의 이해,이 강의는 사회학의 개론을 다루는 교양 강의입니다별도 선수과목은 요구하지 않으며 사...,"[강의, 사회학, 개론, 다루는, 교양, 강의, 별도, 선수, 요구, 하지, 않으며..."
6,수학사,구석기 시대부터 기원전 수천 년에 걸친 문명의 탄생과 고대로부터 현대에 이르기까지 ...,"[구석기, 시대, 기원전, 수천, 걸친, 문명, 탄생, 고대로, 현대, 인간, 문화..."
7,편미분방정식,편미분 방정식은 자연현상을 수학적으로 기술한 형태중 편미분을 포함한 경우에 해당하는...,"[편미분, 방정식, 자연현상, 수학, 기술, 형태, 편미분, 포함, 경우, 해당, ..."
8,수학논리및논술?,본 수업에서는 학교 수학에 포섭된 수학 논리 논술과 관련한 선행 논의들을 검토하고 ...,"[수업, 학교, 수학, 포섭, 수학, 논리, 논술, 선행, 논의, 검토, 중등, 학..."


In [223]:
data_li = data.tagged.values.tolist()
data_li

[['국문과',
  '개설',
  '문학',
  '문학',
  '기본',
  '이해',
  '국문학',
  '흐름',
  '개괄',
  '전교생',
  '대상',
  '동시',
  '문학',
  '현상',
  '이해',
  '한다',
  '동시',
  '문학',
  '매주',
  '질문',
  '형식',
  '탐구',
  '주제',
  '던지고',
  '학생',
  '토론',
  '교수',
  '강의',
  '수업',
  '진행',
  '된다'],
 ['한국',
  '미술사',
  '국회',
  '화사',
  '논의',
  '되는',
  '걸작',
  '중점',
  '강의',
  '조선시대',
  '세기',
  '이르는',
  '시기',
  '창작',
  '전통',
  '회화',
  '조형',
  '특성',
  '구분',
  '담긴',
  '한국',
  '미감',
  '살펴봅니다',
  '이렇듯',
  '한국',
  '미술',
  '걸작',
  '명명',
  '회화',
  '역사',
  '맥락',
  '고찰',
  '하면',
  '시대',
  '영위',
  '예술가',
  '발자취',
  '남긴',
  '예술',
  '독창',
  '수업'],
 ['수업',
  '서양철학',
  '사의',
  '전반',
  '문제',
  '흐름',
  '기본',
  '이해',
  '돕는',
  '이를',
  '고대',
  '현대',
  '유럽',
  '철학',
  '대표',
  '몇몇',
  '철학자',
  '선별',
  '철학자',
  '핵심',
  '사유',
  '주요',
  '논점',
  '다루어',
  '한다',
  '강의',
  '철학자',
  '논의',
  '전통',
  '형이상학',
  '근본',
  '개념',
  '주제',
  '부각시키는',
  '가운데',
  '강의',
  '후반',
  '전통',
  '형이상학적',
  '사유',
  '벗어난',
  '현대',
  '다양한',
  '철학',
  '경향',
  '주제',
  '대표',
  '철학자',
  '기본'

In [224]:
data['final_text'] = data['tagged'].apply(lambda x : " ".join(x))

In [225]:
print(data)

         강의명                                               수업목표  \
0     문학의 이해  국문과에 개설된 문학이란 무엇인가 과이 문학에 대한 기본적 이해와 한국문학의 흐름에...   
1      한국미술사  한국미술사에서는 한국회화사에서 논의되는 걸작을 중점적으로 강의합니다 특히 조선시대부...   
2   서양철학의 이해  본 수업의 목표는 서양철학사의 전반적인 문제와 흐름에 대한 기본적인 이해를 돕는 것...   
3  현대사회와정신건강  현대사회에서 신체적 건강뿐만 아니라 정신적 건강의 중요성이 더욱 부각되고 있는 시점...   
4      언어의세계  인간은 언어적 존재다 인간의 핵심에 언어가 있다 그런데 인간만 언어를 갖고 있는 것...   
5    사회학의 이해  이 강의는 사회학의 개론을 다루는 교양 강의입니다별도 선수과목은 요구하지 않으며 사...   
6        수학사  구석기 시대부터 기원전 수천 년에 걸친 문명의 탄생과 고대로부터 현대에 이르기까지 ...   
7     편미분방정식  편미분 방정식은 자연현상을 수학적으로 기술한 형태중 편미분을 포함한 경우에 해당하는...   
8   수학논리및논술?  본 수업에서는 학교 수학에 포섭된 수학 논리 논술과 관련한 선행 논의들을 검토하고 ...   

                                              tagged  \
0  [국문과, 개설, 문학, 문학, 기본, 이해, 국문학, 흐름, 개괄, 전교생, 대상...   
1  [한국, 미술사, 국회, 화사, 논의, 되는, 걸작, 중점, 강의, 조선시대, 세기...   
2  [수업, 서양철학, 사의, 전반, 문제, 흐름, 기본, 이해, 돕는, 이를, 고대,...   
3  [사회, 신체, 건강, 아니라, 정신, 건강, 중요성, 더욱, 부각, 되고, 있는,...   
4  [인간, 언어, 존재, 인간, 핵심, 언어, 인간, 언어, 갖고, 있는, 세계, 전

In [226]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=50000, min_df=3, analyzer='word')
X = vectorizer.fit_transform(data['final_text'])

lda_model = LatentDirichletAllocation(n_components=2, learning_method='online', random_state=777, max_iter=1)
lda_top = lda_model.fit_transform(X)
print(lda_model.components_)
print(lda_model.components_.shape)

terms = vectorizer.get_feature_names()

def get_topics(components, feature_names, n=20):
    for idx, topic in enumerate(components):
        print("Topic %d :" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n -1:-1]])
        
get_topics(lda_model.components_, terms)

[[0.99000369 0.88835472 1.10845374 1.04180667 0.84193772 1.23234295
  1.05621991 1.34106593 0.97443876 1.04212782 1.14802361 0.97369459
  1.04303622 1.04795225 1.33006625]
 [1.18274958 1.14941505 1.0277917  1.03162756 0.85900251 0.93282376
  1.10584951 1.11100702 1.06523395 0.86922418 0.86884848 1.10729653
  0.92030703 0.79747827 1.05570277]]
(2, 15)
Topic 1 : [('수학', 1.34), ('한다', 1.33), ('사회', 1.23), ('있는', 1.15), ('논의', 1.11), ('수업', 1.06), ('하는', 1.05), ('토대', 1.04), ('인간', 1.04), ('다양한', 1.04), ('강의', 0.99), ('이해', 0.97), ('주제', 0.97), ('기본', 0.89), ('문제', 0.84)]
Topic 2 : [('강의', 1.18), ('기본', 1.15), ('수학', 1.11), ('주제', 1.11), ('수업', 1.11), ('이해', 1.07), ('한다', 1.06), ('다양한', 1.03), ('논의', 1.03), ('사회', 0.93), ('토대', 0.92), ('인간', 0.87), ('있는', 0.87), ('문제', 0.86), ('하는', 0.8)]


c:\Users\dmb08\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [227]:
from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric
from gensim import corpora
from gensim.models.callbacks import PerplexityMetric

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [228]:
dictionary = corpora.Dictionary(processed_data)

2022-07-29 22:55:58,907 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2022-07-29 22:55:58,910 : INFO : built Dictionary<287 unique tokens: ['강의', '개괄', '개설', '교수', '국문과']...> from 9 documents (total 463 corpus positions)
2022-07-29 22:55:58,911 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<287 unique tokens: ['강의', '개괄', '개설', '교수', '국문과']...> from 9 documents (total 463 corpus positions)", 'datetime': '2022-07-29T22:55:58.911820', 'gensim': '4.2.0', 'python': '3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22000-SP0', 'event': 'created'}


In [229]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [230]:
from sklearn.feature_extraction.text import CountVectorizer

cv = TfidfVectorizer()
# 파리미터 삭제함
review_cv = cv.fit_transform(data['final_text']) 

In [231]:
#Checking the sparsity of the data
# Materialize the sparse data
data_dense = review_cv.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("sparsity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

sparsity:  13.743708865660084 %


In [232]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = TfidfVectorizer(min_df=3, analyzer='word', max_features=50000)
# 파라미터 삭제함
data_vectorized = vectorizer.fit_transform(data['final_text']) 

In [233]:
from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric
from gensim import corpora
from gensim.models.callbacks import PerplexityMetric

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [234]:
data

,강의명,수업목표,tagged,final_text
0,문학의 이해,국문과에 개설된 문학이란 무엇인가 과이 문학에 대한 기본적 이해와 한국문학의 흐름에...,"[국문과, 개설, 문학, 문학, 기본, 이해, 국문학, 흐름, 개괄, 전교생, 대상...",국문과 개설 문학 문학 기본 이해 국문학 흐름 개괄 전교생 대상 동시 문학 현상 이...
1,한국미술사,한국미술사에서는 한국회화사에서 논의되는 걸작을 중점적으로 강의합니다 특히 조선시대부...,"[한국, 미술사, 국회, 화사, 논의, 되는, 걸작, 중점, 강의, 조선시대, 세기...",한국 미술사 국회 화사 논의 되는 걸작 중점 강의 조선시대 세기 이르는 시기 창작 ...
2,서양철학의 이해,본 수업의 목표는 서양철학사의 전반적인 문제와 흐름에 대한 기본적인 이해를 돕는 것...,"[수업, 서양철학, 사의, 전반, 문제, 흐름, 기본, 이해, 돕는, 이를, 고대,...",수업 서양철학 사의 전반 문제 흐름 기본 이해 돕는 이를 고대 현대 유럽 철학 대표...
3,현대사회와정신건강,현대사회에서 신체적 건강뿐만 아니라 정신적 건강의 중요성이 더욱 부각되고 있는 시점...,"[사회, 신체, 건강, 아니라, 정신, 건강, 중요성, 더욱, 부각, 되고, 있는,...",사회 신체 건강 아니라 정신 건강 중요성 더욱 부각 되고 있는 시점 수업 정신건강 ...
4,언어의세계,인간은 언어적 존재다 인간의 핵심에 언어가 있다 그런데 인간만 언어를 갖고 있는 것...,"[인간, 언어, 존재, 인간, 핵심, 언어, 인간, 언어, 갖고, 있는, 세계, 전...",인간 언어 존재 인간 핵심 언어 인간 언어 갖고 있는 세계 전체 언어 세계 창세기 ...
5,사회학의 이해,이 강의는 사회학의 개론을 다루는 교양 강의입니다별도 선수과목은 요구하지 않으며 사...,"[강의, 사회학, 개론, 다루는, 교양, 강의, 별도, 선수, 요구, 하지, 않으며...",강의 사회학 개론 다루는 교양 강의 별도 선수 요구 하지 않으며 사회학 사회과학 기...
6,수학사,구석기 시대부터 기원전 수천 년에 걸친 문명의 탄생과 고대로부터 현대에 이르기까지 ...,"[구석기, 시대, 기원전, 수천, 걸친, 문명, 탄생, 고대로, 현대, 인간, 문화...",구석기 시대 기원전 수천 걸친 문명 탄생 고대로 현대 인간 문화 토대 되어 서양 수...
7,편미분방정식,편미분 방정식은 자연현상을 수학적으로 기술한 형태중 편미분을 포함한 경우에 해당하는...,"[편미분, 방정식, 자연현상, 수학, 기술, 형태, 편미분, 포함, 경우, 해당, ...",편미분 방정식 자연현상 수학 기술 형태 편미분 포함 경우 해당 하는 방정식 한다 많...
8,수학논리및논술?,본 수업에서는 학교 수학에 포섭된 수학 논리 논술과 관련한 선행 논의들을 검토하고 ...,"[수업, 학교, 수학, 포섭, 수학, 논리, 논술, 선행, 논의, 검토, 중등, 학...",수업 학교 수학 포섭 수학 논리 논술 선행 논의 검토 중등 학교 교수 학적 상황 적...


In [235]:
t_text = [i for i in data['tagged']]
dictionary = corpora.Dictionary(t_text)
dictionary.filter_extremes(no_below=2, no_above=0.5)
corpus = [dictionary.doc2bow(text) for text in t_text]

2022-07-29 22:55:59,496 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2022-07-29 22:55:59,498 : INFO : built Dictionary<287 unique tokens: ['강의', '개괄', '개설', '교수', '국문과']...> from 9 documents (total 463 corpus positions)
2022-07-29 22:55:59,499 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<287 unique tokens: ['강의', '개괄', '개설', '교수', '국문과']...> from 9 documents (total 463 corpus positions)", 'datetime': '2022-07-29T22:55:59.499094', 'gensim': '4.2.0', 'python': '3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22000-SP0', 'event': 'created'}
2022-07-29 22:55:59,501 : INFO : discarding 249 tokens: [('강의', 5), ('개괄', 1), ('개설', 1), ('국문과', 1), ('국문학', 1), ('대상', 1), ('던지고', 1), ('동시', 1), ('된다', 1), ('매주', 1)]...
2022-07-29 22:55:59,502 : INFO : keeping 38 tokens which were in no less than 2 and no more than 4 (=50.0%) documents
2022-07-29 22:55:59,503 : INFO : resulting dictionary: Dictionary<38 unique toke

In [236]:
#모델링
## 토픽 수 여기서 설정
num_topics = 2
chunksize = 2000
passes = 20
iterations = 400
eval_every = None

temp = dictionary[0]
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every)


2022-07-29 22:55:59,599 : INFO : using autotuned alpha, starting with [0.5, 0.5]
2022-07-29 22:55:59,600 : INFO : using serial LDA version on this node
2022-07-29 22:55:59,601 : INFO : running online (multi-pass) LDA training, 2 topics, 20 passes over the supplied corpus of 9 documents, updating model once every 9 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
2022-07-29 22:55:59,603 : INFO : PROGRESS: pass 0, at document #9/9
2022-07-29 22:55:59,630 : INFO : optimized alpha [0.5447132, 0.6537246]
2022-07-29 22:55:59,631 : INFO : topic #0 (0.545): 0.076*"사회" + 0.076*"인간" + 0.072*"있는" + 0.047*"다양한" + 0.042*"기본" + 0.042*"이를" + 0.040*"주제" + 0.037*"핵심" + 0.035*"기초" + 0.031*"전통"
2022-07-29 22:55:59,632 : INFO : topic #1 (0.654): 0.087*"수학" + 0.061*"인간" + 0.058*"하는" + 0.042*"논의" + 0.041*"공부" + 0.041*"역사" + 0.039*"특성" + 0.036*"사회" + 0.030*"영향" + 0.030*"토대"
2022-07-29 22:55:59,632 : INFO : topic diff=0.609013, rho=1.000000
2022-07-29

In [237]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

Average topic coherence: -8.4674.
[([(0.102209896, '인간'),
   (0.0699462, '사회'),
   (0.06994448, '있는'),
   (0.0484213, '다양한'),
   (0.04840032, '기본'),
   (0.04839777, '주제'),
   (0.03766324, '이를'),
   (0.037661113, '핵심'),
   (0.03765989, '기초'),
   (0.026903955, '전통'),
   (0.02690286, '문제'),
   (0.026902618, '근본'),
   (0.026902428, '현대'),
   (0.026902057, '하고자'),
   (0.026899992, '고대'),
   (0.026899925, '아니라'),
   (0.026859304, '흐름'),
   (0.026855223, '현상'),
   (0.016141824, '토대'),
   (0.016141696, '논의')],
  -7.940390550683748),
 ([(0.110171646, '수학'),
   (0.06611013, '하는'),
   (0.051414743, '논의'),
   (0.051413648, '공부'),
   (0.051413417, '역사'),
   (0.05141241, '특성'),
   (0.036727663, '사회'),
   (0.03672485, '영향'),
   (0.03672467, '토대'),
   (0.036723148, '시대'),
   (0.036722783, '세기'),
   (0.022444747, '학생'),
   (0.022442762, '교수'),
   (0.022065403, '인간'),
   (0.022040114, '있는'),
   (0.022038264, '마지막'),
   (0.022036253, '하고자'),
   (0.022035742, '현대'),
   (0.022035599, '주요'),
   (0.022035573

In [238]:
import pickle
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

## LDA 시각화 html

In [239]:
lda_visualization = gensimvis.prepare(model, corpus, dictionary, sort_topics=False)
pyLDAvis.save_html(lda_visualization, 'C:\\Users\\dmb08\\Desktop\\kucc_ycc_hack\\test_lda_vis44.html')

c:\Users\dmb08\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


In [240]:
for i, topic_list in enumerate(model[corpus]):
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(0, 0.990902)]
1 번째 문서의 topic 비율은 [(0, 0.010989884), (1, 0.9890101)]
2 번째 문서의 topic 비율은 [(0, 0.99708694)]
3 번째 문서의 topic 비율은 [(0, 0.99708694)]
4 번째 문서의 topic 비율은 [(0, 0.9967467)]
5 번째 문서의 topic 비율은 [(0, 0.995755)]
6 번째 문서의 topic 비율은 [(1, 0.9964811)]
7 번째 문서의 topic 비율은 [(1, 0.9944493)]
8 번째 문서의 topic 비율은 [(1, 0.99444926)]


In [241]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [242]:
topictable = make_topictable_per_doc(model, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,0.0,0.9909,"[(0, 0.990902)]"
1,1,1.0,0.9890,"[(0, 0.010989884), (1, 0.9890101)]"
2,2,0.0,0.9971,"[(0, 0.99708694)]"
3,3,0.0,0.9971,"[(0, 0.99708694)]"
4,4,0.0,0.9967,"[(0, 0.9967467)]"
5,5,0.0,0.9958,"[(0, 0.995755)]"
6,6,1.0,0.9965,"[(1, 0.9964811)]"
7,7,1.0,0.9944,"[(1, 0.9944493)]"
8,8,1.0,0.9944,"[(1, 0.99444926)]"


In [243]:
# 해당 토픽에 속하는 강의명 출력
topic0 = topictable[topictable['가장 비중이 높은 토픽'] == 0]
topic0_li = topic0['문서 번호'].values.tolist()

topic1 = topictable[topictable['가장 비중이 높은 토픽'] == 1]
topic1_li = topic1['문서 번호'].values.tolist()

print('topic0')
for i in topic0_li:
  print(data.iloc[i,0])

print('topic1')
for i in topic1_li:
  print(data.iloc[i,0])

topic0
문학의 이해
서양철학의 이해
현대사회와정신건강
언어의세계
사회학의 이해
topic1
한국미술사
수학사
편미분방정식
수학논리및논술?
